In [ ]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'C:\Users\ruben\Desktop\GEM\geoloc2\Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:\Users\ruben\yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
from clasificacion_chinchetas import *
import tqdm
import numpy as np
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import sys
from shapely.geometry import mapping
from shapely import geometry
import cv2
import numpy as np

## Raster y shape deteccion

In [ ]:
raster=r"C:\Users\ruben\Desktop\GEM\Geo\Rasters\Naucalpan_chinchetas.tif"
src=rasterio.open(raster)

deteccion=r"C:\Users\ruben\Desktop\GEM\Geo\Salidas\Chinchetas\Chinchetas_naucalpan_yolo.shp"
deteccion_shp=gpd.read_file(deteccion)


In [ ]:
deteccion_shp=deteccion_shp.dropna().reset_index(drop=True)
angulo_manzana=[]
for manzana in range(len(deteccion_shp)):
    proyecciones1=mapping(deteccion_shp['geometry'][manzana]).get('coordinates')
    angulos=[]
    d=[]
    poly=pd.DataFrame(proyecciones1[0])
    for point in range(1,len(poly)):
        d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
        angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    angulo_manzana.append(angulos[d.index(max(d))])
deteccion_shp['angulo']=angulo_manzana
deteccion_shp1=ampliar_shape(deteccion_shp,factor_ampliacion=1.25)

## Clasificar casas y/o terrenos (Alexnet)

##### Pesos y diccionario Alexnet

In [ ]:
weights=r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_mod11.pth'
num_classes=5
diciconario={0: 'casas', 1: 'en_construccion', 2: 'establecimiento', 3: 'multivivienda', 4: 'terreno_baldio'}
model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)


# weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell11.pth"
# num_classes=6
# diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
# model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

#### Clasificación

In [ ]:
import tqdm
import cv2
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    
    imagen_1=rotate_image(stacked_images,deteccion_shp["angulo"][i],reshape=False)
    clase,imagen=model_class.predict_image(imagen_1)
    
#     cv2.imshow("im",imagen)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    deteccion_shp.loc[i,"clase_n"]=clase

## Clasificar chinchetas (Matching template)

#### Plantillas

In [ ]:
# No borrar path compañeros

path_ruben = r'C:\Users\ruben\Desktop\GEM\Geo\Bases\Plantillas'
path_hector = '/home/hector/Documentos/Infis/Geo/Data/Plantillas'
dict_gris = get_dict_plantilla_gris(path_ruben,True,(30,30))

#### Clasificación

In [ ]:
import tqdm
import cv2
for i,polygo in tqdm.tqdm(enumerate(deteccion_shp1["geometry"]),total=len(deteccion_shp1)):
    det="a"
    try:
        shapes=[mapping(polygo)]
    except:
        continue
#     out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    four_images=[array[2],array[1],array[0]]
    stacked_images = np.stack(four_images, axis=-1)
    
    imagen_1=rotate_image(stacked_images,0,reshape=False)

    # clasificacion
    clase_aux, n_aux, umbral_aux  = iter_umbral_fn(imagen_1, dict_gris,n=100,salto_n=10,
                                                    umbral=0.94,min_umbral=0.8)
    # cv2.imshow('df_aux',imagen_1)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    # print(clase_aux) 
    
    # Revisamos la mayor confusion que se da ... areas_verdes ~ establecimiento_google
    if clase_aux in ['a_verdes','establecimiento_google']:
        clase_aux = areasv_vs_estabgoogle(imagen_1, dict_gris)    
  
    deteccion_shp.loc[i,"clase_chincheta"] = clase_aux
    deteccion_shp.loc[i,"umbral"] = umbral_aux
    deteccion_shp.loc[i,"n_resize"] = n_aux

In [ ]:
deteccion_shp['clase_chincheta'] = deteccion_shp['clase_chincheta'].astype(str)
deteccion_shp['umbral'] = deteccion_shp['umbral'].astype(float) 
deteccion_shp['n_resize'] = deteccion_shp['n_resize'].astype(int)
deteccion_shp.drop(columns=['centroid']).to_file(r"C:\Users\ruben\Desktop\GEM\Geo\Salidas\Chinchetas\Chinchetas_clasif_naucalpan.shp")

In [ ]:
def cortar_deteccion_municipio(shp_municipio,shp_deteccion,output_path,buffer=20):
    """
    Funcion para cortar las detecciones unicamente dentro de municipo 
    """
    municipio=gpd.read_file(shp_municipio)
    municipio=municipio.to_crs(3857)
    municipio_r=cascaded_union(municipio["geometry"].buffer(buffer))
    deteccion=gpd.read_file(shp_deteccion)
    deteccion=deteccion.to_crs(3857)
    deteccionsobremun=deteccion[[municipio_r.contains(x) for x in deteccion["geometry"].centroid]]
    deteccionsobremun.to_file(output_path)  